## Introduction


Leverage pre-trained model (ResNet50) to build image classifiers.

<a id='item31'></a>

## Import Libraries and Packages

In [1]:
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense

In [0]:
from keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input

<a id='item32'></a>

## Download Data

In [4]:
## get the data
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week3.zip

--2020-03-07 19:36:11--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week3.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261482368 (249M) [application/zip]
Saving to: ‘concrete_data_week3.zip’

concrete_data_week3 100%[===================>] 249.37M  30.6MB/s    in 8.3s    

2020-03-07 19:36:20 (30.2 MB/s) - ‘concrete_data_week3.zip’ saved [261482368/261482368]



In [ ]:
!unzip concrete_data_week3.zip

<a id='item33'></a>

## Define Global Constants

In [0]:
num_classes = 2

image_resize = 224

batch_size_training = 100
batch_size_validation = 100

<a id='item34'></a>

## Construct ImageDataGenerator Instances

In [0]:
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

Use the *flow_from_directory* method to get the training images as follows:

In [8]:
train_generator = data_generator.flow_from_directory(
    'concrete_data_week3/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

Found 30001 images belonging to 2 classes.


Use the *flow_from_directory* method to get the validation images and assign the result to **validation_generator**.

In [9]:
## Type your answer here
validation_generator = data_generator.flow_from_directory(
    'concrete_data_week3/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_validation,
    class_mode='categorical')





Found 10001 images belonging to 2 classes.


<a id='item35'></a>

## Build, Compile and Fit Model

In [10]:
model = Sequential()

Add the ResNet50 pre-trained model to out model. We don't want to include the top layer or the output layer of the pre-trained model. We actually want to define our own output layer and train it so that it is optimized for our image dataset. In order to leave out the output layer of the pre-trained model, we will use the argument *include_top* and set it to **False**.

In [11]:
model.add(ResNet50(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))












94658560/94653016 [==============================] - 2s 0us/step


Define the output layer as a **Dense** layer, that consists of two nodes and uses the **Softmax** function as the activation function.

In [12]:
model.add(Dense(num_classes, activation='softmax'))

In [13]:
model.layers

In [14]:
model.layers[0].layers

Since the ResNet50 model has already been trained, then tell the model not to bother with training the ResNet part, but to train only the dense output layer.

In [0]:
model.layers[0].trainable = False

In [16]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 4098      
Total params: 23,591,810
Trainable params: 4,098
Non-trainable params: 23,587,712
_________________________________________________________________


Compile the model using the **adam** optimizer.

In [17]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Define steps and epochs

In [0]:
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 2

Train the model

In [19]:
fit_history = model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/2
301/301 [==============================] - 205s 680ms/step - loss: 0.0507 - acc: 0.9812 - val_loss: 0.2429 - val_acc: 0.9025
Epoch 2/2
301/301 [==============================] - 195s 647ms/step - loss: 0.0151 - acc: 0.9964 - val_loss: 0.2078 - val_acc: 0.9188


Save the model

In [0]:
model.save('classifier_resnet_model.h5')

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).